In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import liana as li

In [2]:
adata = sc.read_h5ad("../kuppe/results/kuppe_processed.h5ad")

/home/dbdimitrov/miniconda3/envs/spiana/lib/python3.10/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.


In [3]:
lrs = li.mt.cellphonedb.by_sample(adata,
                                  groupby='cell_type_original',
                                  use_raw=False,
                                  sample_key='sample',
                                  expr_prop=0.05,
                                  return_all_lrs=True,
                                  verbose=True,
                                  n_perms=1000,
                                  resource_name='consensus',
                                  inplace=False
                                  )

Now running: CK376: 100%|██████████| 28/28 [06:34<00:00, 14.09s/it]


In [4]:
lrs_sub = lrs[(lrs['target']=='MY') & (lrs['cellphone_pvals'] < 0.05)]

In [5]:
lrs_agg = lrs_sub.groupby(['ligand_complex', 'receptor_complex', 'source', 'target'])

In [6]:
# # apply z-score
# lrs_agg = lrs_agg.apply(lambda x: (x['lr_means'] - x['lr_means'].mean()) / x['lr_means'].std())
# lrs_agg = lrs_agg.reset_index()

In [7]:
lrs_agg = lrs_agg.mean().reset_index()

/tmp/ipykernel_3589833/3638309208.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.


In [8]:
part = lrs_agg.sort_values("lr_means", ascending=False).sort_values("lr_means").head(15)

In [9]:
part

,ligand_complex,receptor_complex,source,target,ligand_means,ligand_props,receptor_means,receptor_props,lrs_to_keep,lr_means,cellphone_pvals
3965,LEFTY1,ACVR2B,MA,MY,0.058889,0.068966,0.074525,0.050074,1.0,0.066707,0.015
1360,CD14,TLR6,CM,MY,0.037528,0.054763,0.101485,0.069592,1.0,0.069507,0.032
4480,NODAL,ACVR1B_ACVR2B,CM,MY,0.034484,0.051276,0.104720,0.069219,1.0,0.069602,0.001
4478,NODAL,ACVR1B_ACVR2A,CM,MY,0.034484,0.051276,0.104720,0.069219,1.0,0.069602,0.001
6228,TNF,TRAF2,MA,MY,0.064632,0.053571,0.074993,0.058036,1.0,0.069812,0.014
2764,GDF11,ACVR1B_ACVR2B,CM,MY,0.036973,0.055509,0.104720,0.069219,1.0,0.070846,0.009
1116,CALM3,PDE1B,CM,MY,0.057694,0.062773,0.084252,0.055325,1.0,0.070973,0.000
4584,OMG,RTN4RL1,CM,MY,0.046230,0.081756,0.096655,0.059805,1.0,0.071443,0.000
6227,TNF,TRAF2,CC,MY,0.068497,0.074074,0.074993,0.058036,1.0,0.071745,0.037
5908,TGFB2,ACVR1B_TGFBR2,CM,MY,0.041605,0.057927,0.104720,0.069219,1.0,0.073162,0.045


In [10]:
gt = li.rs.explode_complexes(part[['ligand_complex', 'receptor_complex']], SOURCE='ligand_complex', TARGET='receptor_complex')

/home/dbdimitrov/miniconda3/envs/spiana/lib/python3.10/site-packages/liana/resource/_reassemble_complexes.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [11]:
gt = np.union1d(gt['ligand_complex'], gt['receptor_complex'])
len(gt)

23

In [12]:
gt

array(['ACVR1B', 'ACVR2A', 'ACVR2B', 'BMP8B', 'BMPR1A', 'BMPR1B', 'CALCB',
       'CALM3', 'CD14', 'GDF11', 'GDF7', 'GPR84', 'LEFTY1', 'NODAL',
       'OMG', 'PDE1B', 'RTN4RL1', 'TGFB2', 'TGFBR2', 'TLR6', 'TNF',
       'TRAF2', 'TSHR'], dtype=object)

### SpatialDM

In [28]:
import os
import scanpy as sc
import liana as li
import pandas as pd

In [11]:
data_dir = os.path.join('..', '..', 'data', 'heart_visium')
dataset_names = [f for f in os.listdir(os.path.join('..', 'kuppe', 'results', 'processed')) if f.endswith('.h5ad')]

In [34]:
lrdatas = {}
res = []

for dataset_name in dataset_names:
   adata = sc.read_h5ad(os.path.join('..', 'kuppe', 'results', 'processed', dataset_name))

   li.mt.lr_bivar(adata,
                  function_name='morans',
                  expr_prop=0.1,
                  n_perms=0,
                  use_raw=False,
                  add_categories=False,
                  mask_negatives=True,
                  )
   lrdata = adata.obsm['local_scores']
   df = lrdata.var
   df['sample'] = dataset_name
   res.append(df)

   lrdatas[dataset_name] = lrdata

In [40]:
ii = pd.concat(res)

In [42]:
ii.to_csv("morans.csv")